### Using manual feature-map

### Naive binarization (works)

In [298]:
%reset -f

from collections import defaultdict
from collections import Counter
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

def remapOnFeatureMap2(data, featureMap2):
    remapOnFeatureMap2 = []
    
    for i, row in enumerate(data):
        newRow = []
        for j, d in enumerate(row):
            if j not in [0,7]:
                if (j,d) in featureMap2:
                    newRow.append(featureMap2[(j,d)])
        remapOnFeatureMap2.append(newRow)
    
    binRemapped2 = np.zeros((len(data), len(featureMap2)), dtype=float)
    for i, row in enumerate(remapOnFeatureMap2):
        for j, d in enumerate(row):
            if j not in [0,7]:
                binRemapped2[i][d] = 1
            else:
                binRemapped2[i][j] = d
        binRemapped2[i][-1] = 1
    return binRemapped2

def removeNA(data):
    for i, row in enumerate(data):
        for j, d in enumerate(row):
            if d=='NA':
                data[i][j] = 0
    return data

def removeNAFromDF(data):
    for col in data.columns:
        if col not in binTrainDF.columns:
            data = data.drop(col, axis = 1)
    print('After dropping extra cols from data ----- size of binTrainDF = {} and dataDF = {}'.format(binTrainDF.shape, data.shape))
    
    for col in binTrainDF.columns:
        if col not in data.columns:
            data[col] = 0
    print('After setting cols in data which are not in train to zero ----- size of binTrainDF = {} and dataDF = {}'.format(binTrainDF.shape, data.shape))
    
    for col in data.columns:
        data[col] = data[col].fillna(data[col].mean())
#         data[col] = data[col].fillna(0)
    
    return data

def writePredictions(ids, preds):
    final = [str(i)+','+str(p) for i,p in zip(ids, preds)]
    with open('submission.csv', 'w') as output:
        output.write('Id,SalePrice\n')
        for x in final:
            output.write(str(x))
            output.write('\n')

if __name__ == '__main__':
    trainFile, devFile, testFile = 'my_train.csv', 'my_dev.csv', 'test.csv'
    
    train = np.array([s.strip().split(',')[1:-1] for s in open(trainFile).readlines()][1:])
    dev = np.array([s.strip().split(',')[1:-1] for s in open(devFile).readlines()][1:])
    test = np.array([s.strip().split(',')[1:] for s in open(testFile).readlines()][1:])
#     print('len of dev {} and len of test {}'.format(len(dev), len(test)))

    header = np.array([s.strip().split(',')[1:-1] for s in open(trainFile).readlines()][0])
#     print([(h,val) for h, val in enumerate(header)])
    
    trainIDs = [s.strip().split(',')[0] for s in open(trainFile).readlines()][1:]
    devIDs = [s.strip().split(',')[0] for s in open(devFile).readlines()][1:]
    testIDs = [s.strip().split(',')[0] for s in open(testFile).readlines()][1:]
    
#     trainY = map(float, [s.strip().split(',')[-1] for s in open(trainFile).readlines()][1:])
    trainY = map(lambda s : np.log(float(s)), [s.strip().split(',')[-1] for s in open(trainFile).readlines()][1:])
    trainY = np.array(list(trainY))
#     devY = map(float, [s.strip().split(',')[-1] for s in open(devFile).readlines()][1:])
    devY = map(lambda s : np.log(float(s)), [s.strip().split(',')[-1] for s in open(devFile).readlines()][1:])
    devY = np.array(list(devY))

    train = removeNA(train)
    dev = removeNA(dev)
    
    featureMap = defaultdict()
    featureRemap = defaultdict()
    featureCategories = defaultdict(list)
    for i, row in enumerate(train):
        for j, d in enumerate(row):
            feature = (j,d)
            if feature not in featureMap:
                featureMap[feature] = len(featureMap)
                featureRemap[len(featureMap)-1] = (feature)
                featureCategories[j].append(d)
    
    print('Len of feature map: {}'.format(len(featureMap)))
    
    binTrain = remapOnFeatureMap2(train, featureMap)
    binDev = remapOnFeatureMap2(dev, featureMap)
    binTest = remapOnFeatureMap2(test, featureMap)
    
    model = LinearRegression().fit(X=binTrain, y=trainY)
    coefs = model.coef_
    intercept = model.intercept_
    preds = model.predict(binDev)
    
#     print('len(coefs): {} coefs: {}'.format(len(coefs), coefs))
#     sortedCoefs = np.argsort(coefs)
#     print('Sorted most positive coefs: {}'.format([featureRemap[coef] for coef in sortedCoefs][:10]))
#     print('Sorted most negative coefs: {}'.format([featureRemap[coef] for coef in sortedCoefs][-10:]))
    print('mse: {}'.format(np.sqrt(mean_squared_error(devY, preds))))
    print('intercept: {}'.format(intercept))
    
#     preds = [np.exp(s) for s in preds]
#     print('predictions after exponentiation: {}'.format(list(zip(preds,np.exp(devY)))))
    ridgeRMSLEs = {}
    
    for a in range(0,6,1):
        tempModel = Ridge(alpha=a)
        tempModel.fit(X=binTrain, y=trainY)
        tempPreds = tempModel.predict(binDev)
        x = np.sqrt(mean_squared_error(devY, tempPreds))
        ridgeRMSLEs[a] = np.sqrt(mean_squared_error(devY, tempPreds))
    minAlpha = min(ridgeRMSLEs, key = ridgeRMSLEs.get)
    
    print('len of dict: {}'.format(ridgeRMSLEs))
    print('alpha: {} RMSLE: {}'.format(minAlpha, ridgeRMSLEs[minAlpha]))

Len of feature map: 7226
mse: 0.15764496505166586
intercept: 12.228006625696883


C:\Users\Sheekha\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:165: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number2.189789e-17
  overwrite_a=False)


len of dict: {0: 0.1576453150263322, 1: 0.14654312060859798, 2: 0.14556438228299906, 3: 0.14511864130894428, 4: 0.1449246011514437, 5: 0.14487745644923597, 6: 0.14492367195232872, 7: 0.14503208290545053, 8: 0.14518314933193102, 9: 0.14536399822351206, 10: 0.14556583717508376, 11: 0.14578249220971806, 12: 0.14600953201066338, 13: 0.1462437195575389, 14: 0.14648265642634006, 15: 0.1467245451723682, 16: 0.14696802660226824, 17: 0.14721206595369923, 18: 0.14745587178150635, 19: 0.14769883720078153, 20: 0.14794049669693832, 21: 0.14818049399793518, 22: 0.1484185578522117, 23: 0.14865448361647496, 24: 0.14888811910557306, 25: 0.14911935363284526, 26: 0.14934810946009192, 27: 0.14957433507184614, 28: 0.14979799986059553, 29: 0.15001908990372329, 30: 0.1502376045776294, 31: 0.1504535538529248, 32: 0.1506669561046389, 33: 0.15087783634502278, 34: 0.15108622479122827, 35: 0.1512921556997002, 36: 0.15149566641548046, 37: 0.1516967965985438, 38: 0.1518955875918394, 39: 0.15209208190251933, 40: 0.1

### Smart Binarization

In [325]:
%reset -f

import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

def writePredictions(ids, preds):
    final = [str(i)+','+str(p) for i,p in zip(ids, preds)]
    with open('submission.csv', 'w') as output:
        output.write('Id,SalePrice\n')
        for x in final:
            output.write(str(x))
            output.write('\n')
            
def getMergedDF(useTest=False):
    df = pd.concat([trainDF, devDF], axis = 0).reset_index(drop=True)
    if useTest:
        df = pd.concat([trainDF, testDF], axis = 0).reset_index(drop=True)
    return df
    
        
def getBinarizedDF(df, fillna = False):
    mergedDF = pd.get_dummies(df, columns=cat)
    if fillna:
        mergedDF = pd.get_dummies(df, columns=cat).fillna(0)        
    return mergedDF

if __name__ == '__main__':
    trainDF, devDF, testDF = pd.read_csv('my_train.csv'), pd.read_csv('my_dev.csv'), pd.read_csv('test.csv')
    trainYDF, devYDF = list(map(lambda s : np.log(s), trainDF['SalePrice'])), list(map(lambda s : np.log(s), devDF['SalePrice']))
    trainIDs, devIDs, testIDs = trainDF['Id'], devDF['Id'], testDF['Id']
    
#     num = [0,2,3,16,17,18,19,33,35,36,37,42,43,44,45,46,47,48,49,50,51,53,55,60,61,65,66,67,69,70,74,75,76]
#     cat = [1,4,5,6,7,8,9,10,11,12,13,14,15,20,21,22,23,24,25,26,27,28,29,30,31,32,34,38,39,40,41,52,54,56,57,58,59,62,63,64,68,71,72,73,77,78]
    
    trainLen = trainDF.shape[0]
    print('train shape: {}'.format(trainDF.shape))
    print('dev shape: {}'.format(devDF.shape))
    print('test shape: {}'.format(testDF.shape))
    
    trainDF.drop(['Id', 'SalePrice'], inplace=True, axis = 1)
    devDF.drop(['Id', 'SalePrice'], inplace=True, axis = 1)
    testDF.drop('Id', inplace=True, axis = 1)
    
    mergedDF = getMergedDF(useTest=False)
#     mergedDF = getMergedDF(useTest=True)
    print('merged shape after dropping Id and SalePrice: {}'.format(mergedDF.shape))
    
    print('------------------------------------------------------------------------')
        
    num = trainDF.select_dtypes(include=np.number).columns
    cat = trainDF.select_dtypes(exclude=np.number).columns
    print('num of numerical: {} num of cat: {} '.format(len(num), len(cat)))
        
    binMergedDF = getBinarizedDF(mergedDF, fillna=True)
    print('binMergedDF shape after get_dummies: {}'.format(binMergedDF.shape))
    
#     print('cols in binMergedDF: {}'.format(list(enumerate(binMergedDF.columns))))
    
    binTrainDF = binMergedDF.iloc[:trainLen,:]
    binDevDF = binMergedDF.iloc[trainLen:,:]
#     binTestDF = binMergedDF.iloc[trainLen:,:]
    
    print('binTrainDF shape: {} binDevDF shape: {}'.format(binTrainDF.shape, binDevDF.shape))
#     print('binTrainDF shape: {} binTestDF shape: {}'.format(binTrainDF.shape, binTestDF.shape))

#     model = LinearRegression().fit(X=binTrainDF, y=trainYDF)
#     preds = model.predict(binDevDF)
#     preds = model.predict(binTestDF)
#     coefs = model.coef_
#     print('Smart Binarization - RMSLE: {}'.format(np.sqrt(mean_squared_error(devYDF, preds))))
#     print('Weight for bias feature: {}'.format(model.intercept_))
    
#     sortedCoefs = np.argsort(coefs)
#     print('argsort coefs: {}'.format([(s,coefs[s]) for s in sortedCoefs]))
#     print('Sorted most positive coefs: {}'.format([coef,featureRemap[coef] for coef in sortedCoefs][:10]))
#     print('Sorted most negative coefs: {}'.format([featureRemap[coef] for coef in sortedCoefs][-10:]))

#     temp = [(i,x) for (i,x) in enumerate(list(zip(np.exp(devYDF),np.exp(preds))))]
#     print('zipping vals: {}'.format(temp))

#     print('testPreds: {}'.format(list(zip(testIDs,np.exp(preds)))))
#     writePredictions(testIDs,np.exp(preds))
    
#     print('------------------------------------------------------------------------')
#     ridgeRMSLEs = {}
    
#     for a in range(0,6,1):
#         tempModel = Ridge(alpha=a)
#         tempModel.fit(X=binTrainDF, y=trainYDF)
#         tempPreds = tempModel.predict(binDevDF)
#         x = np.sqrt(mean_squared_error(devYDF, tempPreds))
#         ridgeRMSLEs[a] = np.sqrt(mean_squared_error(devYDF, tempPreds))
#     minAlpha = min(ridgeRMSLEs, key = ridgeRMSLEs.get)

    ridgeModel = Ridge(alpha=5)
    ridgeModel.fit(X=binTrainDF, y=trainYDF)
    ridgePreds = ridgeModel.predict(binDevDF)
    print('RMSLE on dev using Ridge regression: {}'.format(np.sqrt(mean_squared_error(devYDF, ridgePreds))))

# #     print('len of dict: {}'.format(ridgeRMSLEs))
#     print('Ridge Regression - alpha: {} RMSLE: {}'.format(minAlpha, ridgeRMSLEs[minAlpha]))
    
#     ridgeModel = Ridge(alpha=5)
#     ridgeModel.fit(X=binTrainDF, y=trainYDF)
#     ridgeTestPreds = ridgeModel.predict(binTestDF)
#     print('Predictions on test using Ridge Regression: {}'.format(list(enumerate(zip(testIDs,np.exp(ridgeTestPreds))))))
#     writePredictions(testIDs,np.exp(ridgeTestPreds))
    
    print('------------------------------------------------------------------------')
    mergedDF = getMergedDF()
    mergedDF['OverallArea'] = mergedDF['LotArea']+mergedDF['TotalBsmtSF']+mergedDF['GrLivArea']+mergedDF['GarageArea']+mergedDF['WoodDeckSF']+mergedDF['OpenPorchSF']+mergedDF['EnclosedPorch']
    mergedDF['OverallArea'] = mergedDF.apply(lambda x : (x['OverallArea'])**2, axis = 1)
    
    binMergedDF = getBinarizedDF(mergedDF, fillna=True)
    print('binMergedDF shape after get_dummies: {}'.format(binMergedDF.shape))
    
    binTrainDF = binMergedDF.iloc[:trainLen,:]
#     binDevDF = binMergedDF.iloc[trainLen:,:]
    binTestDF = binMergedDF.iloc[trainLen:,:]
    
#     print('binTrainDF shape: {} binDevDF shape: {}'.format(binTrainDF.shape, binDevDF.shape))
    print('binTrainDF shape: {} binTestDF shape: {}'.format(binTrainDF.shape, binTestDF.shape))
    
#     ridgeModel = Ridge(alpha=5)
#     ridgeModel.fit(X=binTrainDF, y=trainYDF)
#     ridgePreds = ridgeModel.predict(binDevDF)
#     print('RMSLE on dev using Ridge regression after squaring OverallArea: {}'.format(np.sqrt(mean_squared_error(devYDF, ridgePreds))))
    
    ridgeModel = Ridge(alpha=5)
    ridgeModel.fit(X=binTrainDF, y=trainYDF)
    ridgeTestPreds = ridgeModel.predict(binTestDF)
    print('Predictions on test using Ridge Regression: {}'.format(list(enumerate(zip(testIDs,np.exp(ridgeTestPreds))))))
    writePredictions(testIDs,np.exp(ridgeTestPreds))

train shape: (1314, 81)
dev shape: (146, 81)
test shape: (1459, 80)
merged shape after dropping Id and SalePrice: (1460, 79)
------------------------------------------------------------------------
num of numerical: 36 num of cat: 43 
binMergedDF shape after get_dummies: (1460, 288)
binTrainDF shape: (1314, 288) binDevDF shape: (146, 288)
RMSLE on dev using Ridge regression: 0.12798847295440965
------------------------------------------------------------------------
binMergedDF shape after get_dummies: (1460, 289)
binTrainDF shape: (1314, 289) binTestDF shape: (146, 289)
Predictions on test using Ridge Regression: [(0, (1461, 127340.14795692769)), (1, (1462, 339365.58037164394)), (2, (1463, 132335.43929382256)), (3, (1464, 146597.9848236789)), (4, (1465, 152569.60210937983)), (5, (1466, 89022.47262214836)), (6, (1467, 166947.24164195277)), (7, (1468, 107605.9499681491)), (8, (1469, 175513.99919883962)), (9, (1470, 116595.93578525534)), (10, (1471, 137297.731795753)), (11, (1472, 186270

C:\Users\Sheekha\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number4.571479e-22
  overwrite_a=True).T
